In [2]:
import pandas as pd
import numpy as np

In [3]:
on_faults = pd.read_pickle('../data/on_faults.pkl')

In [4]:
on_faults = on_faults[on_faults['active'] == True]

So we need all readouts of codes other than the breakdown down within a time period of 1 week before the breakdown code occured. 

WE will have to get some sort of aggregate of events occuring up to the derate and then make new binary features of weather or not they occured. Then our result is whether or not the truck derates.  

Each column of the df will be time periods:

7-6 days prior total # of non derate codes (ie not 5246)

6-5 '''

etc.

In [5]:
window_size = pd.Timedelta('1D')

In [6]:
df = on_faults.set_index('EventTimeStamp').sort_index(ascending = False)

Make dummy columns for category of spn, get_dummies function

In [7]:
df = (pd.get_dummies(data = df, columns =['spn'])
        .filter(regex='^(spn|EquipmentID)'))

In [8]:
# derate_times = df[df['spn_5246'] == 1][['EquipmentID', 'spn_5246']] 

In [9]:
# agg_dict = {col: 'sum' for col in df.columns if (col != 'EquipmentID')}

In [10]:
# agg_dict['EquipmentID'] = lambda x : x[-1]

In [11]:
rolling = (df
            .groupby(by = 'EquipmentID')
            .rolling(window = '1D')
            .sum()         

)

In [12]:
rolling['spn_5246'].sort_values(ascending=False)

EquipmentID  EventTimeStamp     
1452         2016-02-09 15:24:56    5.0
1535         2016-01-11 12:12:52    5.0
             2016-01-11 12:12:53    5.0
1528         2017-02-25 07:30:09    5.0
             2017-02-25 07:30:09    5.0
                                   ... 
1610         2017-11-06 09:04:06    0.0
             2017-11-06 11:02:00    0.0
             2017-11-06 14:00:59    0.0
             2017-11-06 14:39:33    0.0
R1764        2015-02-25 06:08:43    0.0
Name: spn_5246, Length: 548848, dtype: float64

2 column ID for nth derate

class imbalance 

In [13]:
ml_data = rolling[rolling['spn_5246'] <= 1]

In [14]:
ml_data['spn_5246'].value_counts()

0.0    546450
1.0      1867
Name: spn_5246, dtype: int64

In [16]:

from sklearn.utils import resample
# Separate majority and minority classes
df_majority = ml_data[ml_data.spn_5246==0]
df_minority = ml_data[ml_data.spn_5246==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=2000,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.spn_5246.value_counts()

0.0    2000
1.0    1867
Name: spn_5246, dtype: int64

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline

from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [18]:
X = df_downsampled.drop(columns='spn_5246')

y = df_downsampled['spn_5246']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.25, random_state = 321)

In [22]:
from lazypredict.Supervised import LazyClassifier

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)

print(models)

100%|██████████| 29/29 [00:37<00:00,  1.28s/it]

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
RandomForestClassifier             0.95               0.95     0.95      0.95   
BaggingClassifier                  0.95               0.95     0.95      0.95   
ExtraTreesClassifier               0.95               0.95     0.95      0.95   
ExtraTreeClassifier                0.94               0.94     0.94      0.94   
DecisionTreeClassifier             0.93               0.93     0.93      0.93   
KNeighborsClassifier               0.93               0.93     0.93      0.93   
XGBClassifier                      0.92               0.92     0.92      0.92   
LGBMClassifier                     0.92               0.92     0.92      0.92   
AdaBoostClassifier                 0.92               0.92     0.92      0.92   
LabelSpreading                     0.92               0.91     0.91      0.91   
LabelPropagation            

In [24]:
pipe = Pipeline(
    steps = [

        ('model', RandomForestClassifier())
      
    ]
    
)

pipe.fit(X_train, y_train)

Pipeline(steps=[('model', RandomForestClassifier())])

In [25]:
confusion_matrix(y_test, pipe.predict(X_test))

array([[477,  35],
       [ 17, 438]])

In [26]:
print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

         0.0       0.97      0.93      0.95       512
         1.0       0.93      0.96      0.94       455

    accuracy                           0.95       967
   macro avg       0.95      0.95      0.95       967
weighted avg       0.95      0.95      0.95       967

